In [2]:
# merge_all.csv 파일을 사용하여 "가격 (원)" 컬럼의 값을 예측하는 모델을 만든다

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import keras




In [3]:
# 데이터 불러오기
df = pd.read_csv('data/merge_all/merge_all.csv')

# 데이터 전처리
# "거래년월일" 컬럼을 인덱스로 설정
df = df.set_index('거래년월일')
# "평균" 컬럼을 배추가격으로 설정
df = df.rename(columns={'평균':'배추가격'})
df.head()

,자동차용 경유 가격 (원),전월비(%),배추가격,거래량
거래년월일,,,,
2000-01-01,583.35,4.0,5600.0,102285.0
2000-01-02,583.35,4.0,5600.0,102285.0
2000-01-03,583.35,4.0,5600.0,102285.0
2000-01-04,583.35,4.0,5600.0,715338.0
2000-01-05,583.35,4.0,5620.0,340568.0


In [4]:
# df 를 시퀀스 데이터로 변환
# 1. 데이터를 7일 단위로 묶는다

# 7일 단위로 묶는 함수
def make_sequence_data(data, window_size):
    x, y = [], []
    for i in range(len(data) - window_size):
        group = data.iloc[i:(i+window_size)]
        x.append(group.values)
        y.append(data.iloc[i+window_size][0])
    return np.array(x), np.array(y)

# 7일 단위로 묶는다
window_size = 7
x, y = make_sequence_data(df, window_size)
print(x.shape, y.shape)

# 2. 데이터를 학습용 데이터와 테스트용 데이터로 나눈다
# 학습용 데이터와 테스트용 데이터를 7:3으로 나눈다
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size
train_x, test_x = np.array(x[0:train_size]), np.array(x[train_size:len(x)])
train_y, test_y = np.array(y[0:train_size]), np.array(y[train_size:len(y)])
print(train_x.shape, test_x.shape)

# 3. 데이터를 정규화한다
# 정규화 함수 dim=3
def normalize(data):
    norm_data = np.zeros(data.shape)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            norm_data[i, j] = (data[i, j] - np.min(data[i, j])) / (np.max(data[i, j]) - np.min(data[i, j]))
    return norm_data

# 정규화
train_x = normalize(train_x)
test_x = normalize(test_x)
train_x.shape, test_x.shape
train_x





(8363, 7, 4) (8363,)
(5854, 7, 4) (2509, 7, 4)


array([[[5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        [5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        [5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        ...,
        [1.70114868e-03, 0.00000000e+00, 1.64902926e-02, 1.00000000e+00],
        [1.99654692e-03, 0.00000000e+00, 1.96294663e-02, 1.00000000e+00],
        [1.07940276e-03, 0.00000000e+00, 1.05564788e-02, 1.00000000e+00]],

       [[5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        [5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        [8.09901389e-04, 0.00000000e+00, 7.82291908e-03, 1.00000000e+00],
        ...,
        [1.99654692e-03, 0.00000000e+00, 1.96294663e-02, 1.00000000e+00],
        [1.07940276e-03, 0.00000000e+00, 1.05564788e-02, 1.00000000e+00],
        [1.34090793e-03, 0.00000000e+00, 1.33454305e-02, 1.00000000e+00]],

       [[5.66429738e-03, 0.00000000e+00, 5.47120189e-02, 1.00000000e+00],
        

In [5]:
# 모델을 생성한다 shape=(None, 7, 4) return_sequences=True
model = Sequential()
model.add(LSTM(128, input_shape=(7,4), return_sequences=True))
#convolutional layer
model.add(keras.layers.Conv1D(64, 3, activation='relu'))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()



Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-22 16:29:05.318993: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-22 16:29:05.319319: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 128)            68096     
                                                                 
 conv1d (Conv1D)             (None, 5, 64)             24640     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 92,865
Trainable params: 92,865
Non-trainable params: 0
____________________________________________________

In [6]:
# 모델을 학습
model.fit(train_x, train_y, epochs=10000000000, batch_size=10000000, verbose=1)

# 저장
model.save('model/predict_price.h5')

Epoch 1/10000000000


2022-12-22 16:29:05.709697: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-22 16:29:06.472150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-22 16:29:06.607677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 1717051.6250
Epoch 2/10000000000


2022-12-22 16:29:06.882439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 47ms/step - loss: 1716751.6250
Epoch 3/10000000000
1/1 [==============================] - 0s 45ms/step - loss: 1716481.8750
Epoch 4/10000000000
1/1 [==============================] - 0s 45ms/step - loss: 1716220.3750
Epoch 5/10000000000
1/1 [==============================] - 0s 45ms/step - loss: 1715959.2500
Epoch 6/10000000000
1/1 [==============================] - 0s 44ms/step - loss: 1715710.1250
Epoch 7/10000000000
1/1 [==============================] - 0s 43ms/step - loss: 1715439.5000
Epoch 8/10000000000
1/1 [==============================] - 0s 44ms/step - loss: 1715143.8750
Epoch 9/10000000000
1/1 [==============================] - 0s 45ms/step - loss: 1714820.6250
Epoch 10/10000000000
1/1 [==============================] - 0s 50ms/step - loss: 1714459.7500
Epoch 11/10000000000
1/1 [==============================] - 0s 51ms/step - loss: 1714055.3750
Epoch 12/10000000000
1/1 [==============================] - 0s 48ms/step - loss: 1713597

KeyboardInterrupt: 

In [ ]:
#  학습 결과를 그래프로 그린다
plt.plot(model.history.history['loss'])
plt.show()

# 실제값
plt.plot(test_y, label='actual')
plt.legend()
plt.show()



In [ ]:
#  모델을 평가한다
#  RMSE
train_predict = model.predict(train_x)
test_predict = model.predict(test_x)
train_score = np.sqrt(mean_squared_error(train_y, train_predict))
test_score = np.sqrt(mean_squared_error(test_y, test_predict))
print('Train Score: %.2f RMSE' % (train_score))
print('Test Score: %.2f RMSE' % (test_score))



In [ ]:
#  예측 결과를 그래프로 그린다
plt.plot(test_y, label='actual')
plt.plot(test_predict, label='prediction')
plt.legend()
plt.show()
